In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [3]:
spark = SparkSession.builder.getOrCreate()

23/08/04 09:08:25 WARN Utils: Your hostname, antoniogm-ThinkPad-T14-Gen-3 resolves to a loopback address: 127.0.1.1; using 192.168.1.23 instead (on interface wlp0s20f3)
23/08/04 09:08:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/04 09:08:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [53]:
df = spark.read.csv('final_data/', header=True)

df = df.withColumn('State', f.col('State').cast('int')) # Al venir de un .csv, se lee como un String por lo que hay que repetir el casteo

### Contar numero de asientos disponibles a la mañana y a la tarde.

In [34]:

df_morning = df.filter(f.expr('hour(DepartureTime) < 12'))
df_afternoon = df.filter(f.expr('hour(DepartureTime) >= 12'))

n_seats_morning = df_morning.groupBy().sum('State').collect()[0][0]
n_seats_afternoon = df_afternoon.groupBy().sum('State').collect()[0][0]

print('Nº asientos disponibles por la mañana: ', n_seats_morning)
print('Nº asientos disponibles por la tarde:  ', n_seats_afternoon)

Nº asientos disponibles por la mañana:  28029
Nº asientos disponibles por la tarde:   20830


### ¿Existe algún journey con más de una portion?

In [57]:
# Contamos el nº de portions por cada journey
df_portions = df.groupBy('journey_id').agg(f.countDistinct('portion_id').alias('n_portions'))
df_portions.show()
df_journeys_result = df_portions.filter(f.col('n_portions') > 1)

print("Journeys con más de un portion: ")
df_journeys_result.show(1000000)

print('Hay',df_journeys_result.count(),' con más de un portion')

+----------+----------+
|journey_id|n_portions|
+----------+----------+
|       125|         1|
|         7|         1|
|        51|         1|
|       124|         1|
|       169|         1|
|        15|         1|
|        54|         1|
|8589934607|         1|
|       155|         1|
|       154|         1|
|       132|         1|
|8589934618|         1|
|       101|         1|
|        11|         1|
|       138|         1|
|        29|         1|
|        69|         1|
|        42|         1|
|       112|         1|
|        73|         1|
+----------+----------+
only showing top 20 rows

Journeys con más de un portion: 
+----------+----------+
|journey_id|n_portions|
+----------+----------+
+----------+----------+

Hay 0  con más de un portion


### Añadir una nueva columna al dataframe existente con el valor entero de la extended availability.

### Eliminar todos los journeys que contengan más de un vuelo (journeys con escala) del dataframe.

In [47]:
df_flights_per_journey = df.groupBy('journey_id').agg(f.count('flight_id').alias('num_flights'))

df_sin_escala = df_flights_per_journey.filter(f.col('num_flights') == 1)

#df_flights_per_journey.show()

df_result = df.join(df_sin_escala, 'journey_id', 'inner') # Elimina las filas donde no coincide el valormclave
